<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat</span>
<span><img src="media/inria-25-alpha.png" /></span>
</div>

# mon premier merge

In [ ]:
# ce sera toujours notre façon de commencer
[ -f scripts/helpers.sh ] && source scripts/helpers.sh

## repartons de la partie 2

si vous avez bien suivi et exécuté ce qui précède,  
vous devez avoir un répertoire `my-first-repo`:

* qui contient 4 commits
* et deux branches `master` et `devel`
* et vous devez être sur la branche devel

In [ ]:
# si nécessaire, vous pouvez remettre le repository en l'état
# 
# pour cela mettez "true" au lieu de ""
# et bien sûr évaluer la cellule

reset=""

if [ -n "$reset" ]; then 
    cd $TOP
    bash $SCRIPTS/2-01-my-first-repo.sh 
    bash $SCRIPTS/2-02-consistency-repo-fs.sh 
fi >& /dev/null

In [ ]:
# si nécessaire, on se place dans le repo git
[ -d my-first-repo ] && cd my-first-repo

pwd

In [ ]:
# vous devez avoir 4 commits, deux branches
# et être sur la branche devel
git l --all

## branche courante

souvenez vous: 

* on avait fait
 * `git checkout -b devel HEAD^^`
* on avait vu que ça avait :
  * créé une branche `devel` 
  * choisi cette branche comme **courante**

vue gitkraken:

![](media/repo-4c-2b.png)

## branches

notion de branche

* une branche correspond uniquement à  
  **une marque posée sur un commit**

* et de plus la branche **courante**  
  suit les **commits** au fur et à mesure

## on committe dans la branche courante

comme nous sommes sur la branche `devel`   

* on va créer le commit avec comme parent  
  le commit #2 (*added LICENSE..*)

* et la branche devel avance d'un cran  
  elle désigne le nouveau commit

In [ ]:
# la branche courante est devel
# du coup si on crée un commit
# maintenant:

echo "dans la branche devel" >> LICENSE
git add LICENSE
git commit -m "le début de la branche devel"

![](media/repo-5c-2b.png)


## changer de branche: `git checkout`

quand on change de branche  
**l'espace de travail est modifié** en conséquence

In [ ]:
# on est  sur devel
ls

In [ ]:
# on va sur master
git checkout master

In [ ]:
# on retrouve tous nos fichiers
ls

In [ ]:
# on retourne sur devel
git checkout devel

In [ ]:
# les fichiers suivent le contenu du commit
ls

On insiste à nouveau : le fait de changer de branche est une opérations invasive, dans ce sens que cela modifie les fichiers présents sur le disque, pour s'aligner avec le changement de commit courant.

## digression : `git diff` entre commits 

`git diff` fonctionne **aussi** entre deux commits :

In [ ]:
git diff master devel

![](media/repo-5c-2b.png)

## mon premier `merge`

dans sa version la plus simple,
`git merge` permet de 'fusionner' deux branches :

In [ ]:
# on se met dans la branche master
git checkout master

# on vérifie
git branch

In [ ]:
# ce merge va 
# créer un commit, donc:

# - il sera créé 
# sur la branche courante
# ici master
# -  il me faut donner
# un message

git merge devel -m "mon premier merge"

In [ ]:
# remarquez le nouveau commit 
# qui est bien sûr
# créé dans la branche courante
git l

![](media/order-0-both.png)

Pour se souvenir des paramètres, comme toujours, le commit qui est (éventuellement) créé l'est **sur la branche courante**; c'est seulement la branche courante qui avance, i.e. lorsqu'on fait `git merge tutu`, la branche `tutu` **n'est pas mmodifiée**.


## mon premier `merge` - comportement

naturellement:

* le merge inclut tout le code contenu dans le point de la fourche
* ainsi que toutes les modifications faites **dans les deux branches**

* nous allons le vérifier en comparant
  * les modifs entre `left` et `master`
  * avec les modifs entre `fork` et `right`
  * qui doivent être identiques
* et symétriquement

![](media/order-1-compare-labels.png)

## rappel - naviguer dans les commits

In [ ]:
git l

on sait déjà désigner `left` et `right` 

* left = `master^`  
  car `X^` est le premier parent de X

* right = `master^2`  
  le second parent

In [ ]:
git l -1 master^

In [ ]:
git l -1 master~2

Pour plus de détails sur les nombreux autres mécanismes qui existent pour naviguer dans les commits, voir aussi https://git-scm.com/book/en/v2/Git-Tools-Revision-Selection

## digression - le plus proche ancêtre commun

`git merge-base` 

In [ ]:
git l --all

In [ ]:
# l'endroit de la 'fourche' c'est
git merge-base master^ master^2

In [ ]:
fork=$(git merge-base master^ master^2)
echo $fork

pas besoin tous les jours, juste utile pour nous pour démontrer ce qui s'est passé dans ce merge

`git merge-base` permet, étant donné deux commits, de trouver le plus proche ancêtre commun aux deux.

C'est comme ça que l'on détermine le SHA-1 du commit où les deux branches se sont séparées.

À noter également, dans ce code `fork` désigne une **variable `bash`**, ça nous évitera de retaper cette phrase un peu barbare

## les pivots du merge

![](media/order-1-compare-labels.png)

In [ ]:
# on va se définir des raccourcis
# pour désigner les 4 points importants

left="master^"

right="master^2"

In [ ]:
git l -1 master

In [ ]:
git l -1 $left

In [ ]:
git l -1 $right

In [ ]:
git l -1 $fork


## vérifions le merge (1)

<span><img src="media/order-1-compare-labels.png" width="120px" />

In [ ]:
git diff $right master

In [ ]:
git diff $fork $left

## vérifions le merge (2)

<span><img src="media/order-1-compare-labels.png" width="120px" />

In [ ]:
git diff $left master

In [ ]:
git diff $fork $right

Le point que l'on veut faire ici est que, après ce merge, les différences entre `left et master` sont identiques à celles entre `fork` et `right`.

## autant de branches qu'on veut

on ne va pas le faire sur ce premier exemple, mais

* on peut créer autant de branches qu'on veut
  * qui partent de où on veut
  
* on peut aussi merger plusieurs branches dans un merge
  * ça s'appelle *octopus* merge
  
* on peut donc parfaitement avoir
  * *n* branches qui partent du même commit
  * et *n* branches qui sont fusionnées dans un seul commit

À titre anecdotique, on peut effectivement merger plus de deux branches, il suffit pour cela de mentionnner plusieurs branches à `git merge`, comme par exemple `git merge b1 b2 b3`, ce qui créera en général un commit avec 4 parents; cette pratique est connue sous le nom de *octopus merge*.

## résumé



* `git merge` crée si nécessaire **un nouveau commit**
  * qui contient les changements 
  * faits dans les branches fusionnées

* pour fusionner deux commits
  * choisir la branche courante
  * qui comme toujours va recevoir le commit
  * désigner le (ou les) commits qu'il faut fusionner

## état

pour pouvoir s'y référer le cas échéant, voici notre repo à ce stade

In [ ]:
git l --all